In [8]:
from numba import jit
import matplotlib.pyplot as plt

import sys; sys.path.insert(0, '..')
from database.dataset import *
from models.bria2014.feature_extraction import feature_instantiator
from mc_candidate_proposal.hough_mc import HoughCalcificationDetection
from metrics.metrics import get_tp_fp_fn

### Preprocessing

#### Check mayimum value in images

In [2]:
# dcm_path = Path('/home/jseia/Desktop/ml-dl/calc-det/data/INbreast Release 1.0/AllDICOMs/')

# max_val = []
# for dcm in dcm_path.iterdir():
#     if dcm.name.endswith('.dcm'):
#         img = sitk.ReadImage(str(dcm))
#         img_array = sitk.GetArrayFromImage(img)
#         max_val.append(img_array.max())

In [3]:
# dcm_path = Path('/home/jseia/Desktop/ml-dl/calc-det/data/INbreast Release 1.0/AllDICOMs/')

# bits_alloc = []
# bits_stored = []
# for dcm in dcm_path.iterdir():
#     if dcm.name.endswith('.dcm'):
#         img = pydicom.dcmread(str(dcm), stop_before_piyels=True)
#         bits_alloc.append(img['BitsAllocated'].value)
#         bits_stored.append(img['BitsStored'].value)
#         break

#### Quatum noise tests

In [4]:
# def T(y, y_max=4095, T_max=65.535):
#     return T_max*np.sqrt(y)/np.sqrt(y_max)

### Detection stage

In [1]:
import cv2
import sys; sys.path.insert(0, '..')

from database.dataset import INBreast_Dataset
from mc_candidate_proposal.hough_mc import HoughCalcificationDetection
from models.bria2014.haar_extractor import HaarFeatureExtractor
from metrics.metrics import get_tp_fp_fn
from pathlib import Path
from tqdm import tqdm
import pandas as pd

db = INBreast_Dataset(
    return_lesions_mask=True,
    level='image',
    max_lesion_diam_mm=1.,
    extract_patches=False,
    extract_patches_method='all',  # 'centered'
    patch_size=256,
    stride=256,
    min_breast_fraction_roi=0.5,
    normalize=None,
    n_jobs=-1,
    partitions=['train', 'val']
)

dehazing_params = {'omega': 0.9, 'window_size': 11, 'radius': 40, 'eps': 1e-5}

hough1_params = {'method': cv2.HOUGH_GRADIENT, 'dp': 1, 'minDist': 20,
                    'param1': 300, 'param2': 8,  'minRadius': 2, 'maxRadius': 20}

hough2_params = {'method': cv2.HOUGH_GRADIENT, 'dp': 1, 'minDist': 20,
                    'param1': 300, 'param2': 10,  'minRadius': 2, 'maxRadius': 20}
back_ext_radius = 50
erosion_iter = 20
erosion_size = 5

path = Path('/home/jseia/Desktop/ml-dl/data/hough')
path.mkdir(exist_ok=True, parents=True)

hd = HoughCalcificationDetection(
    dehazing_params, back_ext_radius,
    path,
    hough1_params, hough2_params,
    erosion_iter=erosion_iter,
    erosion_size=erosion_size
)

BASE_PATH = Path('../../../data_rois/')

idx = 0
case = db[idx]
image = case['img']
image_id = db.df.iloc[idx].img_id
radiouses = case['radiuses']
true_bboxes = db[idx]['lesion_bboxes']

_, h2_circles = hd.detect(
    image, image_id, load_processed_images=True, hough2=True)
tp, fp, fn, gt_d, close_fp = get_tp_fp_fn(true_bboxes, radiouses, h2_circles, 7, 0.0)

del h2_circles

haarfe = HaarFeatureExtractor(14)
tp_features = haarfe.extract_features(image, tp)
tp_features['label'] = 'tp'
# logging.info('Extracting features from FP')
fp_features = haarfe.extract_features(
    image, fp, haarfe.integral_image, haarfe.diagintegral_image)
fp_features['label'] = 'fp'

features = pd.concat([tp_features, fp_features], ignore_index=True)
del fp_features, tp_features

db_filename = BASE_PATH/'haar_features'/f'{image_id}_tp.fth'
db_filename.parent.mkdir(exist_ok=True, parents=True)
features.to_feather(db_filename)

INFO:numexpr.utils:NumExpr defaulting to 8 threads.


In [3]:
len(haarfe.features_r)

2421

### feature set

Old things

In [19]:
# class Dataset:
#     def __init__(
#         self, pos_df: pd.DataFrame, neg_df: pd.DataFrame,
#         kr: float = 5, normalize: bool = False, seed: int = 42
#     ):
#         self.pos_df = pos_df
#         self.neg_df = neg_df
#         self.kr = kr
#         self.nomalize = normalize
#         self.seed = seed

#         self.train_discarded_files = []
#         self.val_discarded_files = []

#         self.train_pos_cases_list = self.pos_df['filename'].sample(
#             frac=0.5, replace=False, random_state=self.seed
#         )
#         self.train_pos_cases_list = self.train_pos_cases_list.tolist()
#         condition = ~self.pos_df.filename.isin(self.train_pos_cases_list)
#         self.val_pos_cases_list = self.pos_df.loc[condition, 'filename'].tolist()

#         self.n_neg = len(self.train_pos_cases_list) * self.kr
#         self.used_neg_files = []


#     def open_img(self, filename: str):
#         # filename = '/home/jseia/Desktop/ml-dl/data_rois/'+filename
#         return cv2.imread(filename, cv2.IMREAD_ANYDEPTH)
    
#     def open_img_normalize(self, filename: str, mean: float, std: float):
#         # TODO: add defaults mean and std
#         return (cv2.imread(filename, cv2.IMREAD_ANYDEPTH) - mean) / std

#     def load_imgs(self, pos_img_files: list, neg_img_files: list):
#         # Imgs/features # TODO: Define if images or features
#         xs = []
#         # TODO: parallelize dataloading with multithread
#         if self.normalize:
#             xs.extend([self.open_img_normalize(f) for f in pos_img_files])
#             xs.extend([self.open_img_normalize(f) for f in neg_img_files])
#         else:
#             xs.extend([self.open_img(f) for f in pos_img_files])
#             xs.extend([self.open_img(f) for f in neg_img_files])
        
#         # Labels
#         ys = np.zeros((len(pos_img_files)+len(neg_img_files)))
#         ys[:len(pos_img_files)] = 1
#         return np.array(xs), ys

#     def get_train_batch(self, discarded_files: list = None):
#         # Define the number of files to sample
#         n_sample = self.n_neg if discarded_files is None else len(discarded_files)
        
#         # Discard rejected files
#         self.train_neg_cases_list = \
#             [fn for fn in self.train_neg_cases_list if fn not in discarded_files]
        
#         # Define the cases from which to sample and sample
#         condition = ~self.neg_df.filename.isin(self.used_neg_files),
#         sample = self.neg_df.loc[condition, 'filename'].sample(
#             n=n_sample, replace=False, random_state=self.seed
#         )
#         sample = sample.tolist()
#         self.train_neg_cases_list.extend(sample)
#         train_files_list = self.train_pos_cases_list + self.train_neg_cases_list

#         # Keep track of history
#         self.used_neg_files.extend(sample)
        
#         # Load the images/features # TODO: Define if images or features
#         xs, ys = self.load_imgs(self.train_pos_cases_list, self.train_neg_cases_list)
        
#         return train_files_list, xs, ys

#     def get_val_batch(self, discarded_files: list = None):
#         # Define the number of files to sample
#         n_sample = self.n_neg if discarded_files is None else len(discarded_files)
        
#         # Discard rejected files
#         self.val_neg_cases_list = \
#             [fn for fn in self.val_neg_cases_list if fn not in discarded_files]
        
#         # Define the cases from which to sample and sample
#         condition = ~self.neg_df.filename.isin(self.used_neg_files),
#         sample = self.neg_df.loc[condition, 'filename'].sample(
#             n=n_sample, replace=False, random_state=self.seed
#         )
#         sample = sample.tolist()
#         self.val_neg_cases_list.extend(sample)
#         val_files_list = self.val_pos_cases_list + self.val_neg_cases_list

#         # Keep track of history
#         self.used_neg_files.extend(sample)
        
#         # Load the images/features  # TODO: Define if images or features
#         xs, ys = self.load_imgs(self.val_pos_cases_list, self.val_neg_cases_list)
        
#         return val_files_list, xs, ys

In [24]:
# neg = pd.read_csv('/home/jseia/Desktop/ml-dl/data_rois/bg.txt', header=None, nrows=100000)[0].tolist()
# pos = pd.read_csv('/home/jseia/Desktop/ml-dl/data_rois/positives/info.dat', sep=' ', header=None)[0].tolist()

In [98]:
# ThresholdPolarity = tp.NamedTuple(
#     'ThresholdPolarity', [('threshold', float), ('polarity', float)]
# )

# ClassifierResult = tp.NamedTuple(
#     'ClassifierResult', [
#         ('threshold', float), ('polarity', int), ('classification_error', float),
#         ('classifier', tp.Callable[[np.ndarray], float])
#     ])

# WeakClassifier = tp.NamedTuple(
#     'WeakClassifier', [
#         ('threshold', float), ('polarity', int), ('alpha', float),
#         ('classifier', tp.Callable[[np.ndarray], float])
#     ])

# PredictionStats = tp.NamedTuple(
#     'PredictionStats', [('tn', int), ('fp', int), ('fn', int), ('tp', int)]
# )


# def prediction_stats(y_true: np.ndarray, y_pred: np.ndarray) -> Tuple[np.ndarray, PredictionStats]:
#     c = confusion_matrix(y_true, y_pred)
#     tn, fp, fn, tp = c.ravel()
#     return c, PredictionStats(tn=tn, fp=fp, fn=fn, tp=tp)

In [87]:
# average_face = np.sum(xs[ys > .5], axis=0) / xs[ys > .5].shape[0]
# average_face = (average_face - average_face.min()) / (average_face.max() - average_face.min())
# plt.figure()
# plt.imshow((average_face), cmap='gray')

# average_face = np.sum(xs[ys < .5], axis=0) / xs[ys < .5].shape[0]
# average_face = (average_face - average_face.min()) / (average_face.max() - average_face.min())
# plt.figure()
# plt.imshow((average_face), cmap='gray')

In [362]:
# import itertools
# #TODO: check cache
# @njit
# def weak_classifier(
#     x: np.ndarray, f: fm.Feature, polarity: float, theta: float
# ) -> float:
#     # TODO: Check if Bria uses theta and polarity
#     return (np.sign((polarity * theta) - (polarity * f(x))) + 1) // 2

# @njit
# def run_weak_classifier(x: np.ndarray, c: WeakClassifier) -> float:
#     return weak_classifier(x=x, f=c.classifier, polarity=c.polarity, theta=c.threshold)

# @njit
# def strong_classifier(x: np.ndarray, weak_classifiers: tp.List[WeakClassifier]) -> int:
#     sum_hypotheses = 0.
#     # sum_alphas = 0.
#     for c in weak_classifiers:
#         sum_hypotheses += c.alpha * run_weak_classifier(x, c)
#     #     sum_alphas += c.alpha
#     # return 1 if (sum_hypotheses >= .5*sum_alphas) else 0
#     return sum_hypotheses


# @njit
# def fast_rank_loss(preds: np.ndarray, labels: np.ndarray):
#     idx = np.argsort(preds)
#     labels = labels[idx]
#     temp = np.cumsum(labels)
#     count = temp[np.where(labels==0)].sum()
#     return count

# @njit
# def get_rank_mat(preds: np.ndarray, labels: np.ndarray):
#     pos_loc = np.where(labels, True, False)
#     pos_preds = preds[pos_loc]
#     neg_preds = preds[~pos_loc]
#     n_pos_preds = len(pos_preds)
#     n_neg_preds = len(neg_preds)
#     pos_mat = pos_preds.repeat(n_neg_preds).reshape(n_pos_preds, n_neg_preds)
#     neg_mat = neg_preds.repeat(n_pos_preds).reshape(n_neg_preds, n_pos_preds).T
#     return np.where(pos_mat < neg_mat, 1, 0)

# # TODO analyse combining this and previous funct
# @njit
# def weight_update(
#     pos_preds: np.ndarray, neg_preds: np.ndarray, weights:np.ndarray, alpha: float
# ):
#     n_pos_preds = len(pos_preds)
#     n_neg_preds = len(neg_preds)
#     pos_mat = pos_preds.repeat(n_neg_preds).reshape(n_pos_preds, n_neg_preds)
#     neg_mat = neg_preds.repeat(n_pos_preds).reshape(n_neg_preds, n_pos_preds).T
#     return weights * np.exp(alpha * np.diff(neg_mat, pos_mat)) / weights.sum()

# @njit
# def wieghted_rank_loss(
#     preds: np.ndarray, labels: np.ndarray, weights: np.ndarray
# ):
#     rank_mat = get_rank_mat(preds, labels)
#     return (weights * rank_mat).sum()

# @njit
# def get_initial_alpha(
#     pos_preds: np.ndarray, neg_preds: np.ndarray, weights: np.ndarray
# ):
#     n_pos_preds = len(pos_preds)
#     n_neg_preds = len(neg_preds)
#     pos_mat = pos_preds.repeat(n_neg_preds).reshape(n_pos_preds, n_neg_preds)
#     neg_mat = neg_preds.repeat(n_pos_preds).reshape(n_neg_preds, n_pos_preds).T
#     weighted_preds = weights * np.diff(neg_mat, pos_mat)
#     return 1/2 * np.ln((1 + weighted_preds) / (1 - weighted_preds))

# @njit
# def get_pos_and_neg_preds_matrices(preds: np.ndarray, labels: np.ndarray):
#     pos_loc = np.where(labels, True, False)
#     pos_preds = preds[pos_loc]
#     neg_preds = preds[~pos_loc]
#     n_pos_preds = len(pos_preds)
#     n_neg_preds = len(neg_preds)
#     pos_mat = \
#         pos_preds.repeat(n_neg_preds).reshape(n_pos_preds, n_neg_preds)
#     neg_mat = \
#         neg_preds.repeat(n_pos_preds).reshape(n_neg_preds, n_pos_preds).T
#     return pos_mat, neg_mat
    

# @njit
# def choose_weak(
#     strong_class_pred: np.ndarray, x: np.ndarray, labels:np.ndarray,
#     weights: np.ndarray, weak_classifiers: tp.List[WeakClassifier]
# ):
#     # strong preds
#     pos_mat_strong, neg_mat_strong = \
#         get_pos_and_neg_preds_matrices(strong_class_pred, labels)
#     acc = []
#     for c in weak_classifiers:
#         weak_pred = run_weak_classifier(x, c)
#         pos_mat_weak, neg_mat_weak = \
#             get_pos_and_neg_preds_matrices(weak_pred, labels)
#         rank_mat = (
#             neg_mat_strong + c.alpha * neg_mat_weak >= 
#             pos_mat_strong + c.alpha * pos_mat_weak
#         )
#         acc.append(rank_mat * weights)
#     return weak_classifiers[np.argmax(acc)]


In [ ]:
@jit
def build_running_sums(ys: np.ndarray, ws: np.ndarray) -> Tuple[float, float, List[float], List[float]]:
    s_minus, s_plus = 0., 0.
    t_minus, t_plus = 0., 0.
    s_minuses, s_pluses = [], []
    
    for y, w in zip(ys, ws):
        if y < .5:
            s_minus += w
            t_minus += w
        else:
            s_plus += w
            t_plus += w
        s_minuses.append(s_minus)
        s_pluses.append(s_plus)
    return t_minus, t_plus, s_minuses, s_pluses


@jit
def find_best_threshold(zs: np.ndarray, t_minus: float, t_plus: float, s_minuses: List[float], s_pluses: List[float]) -> ThresholdPolarity:
    min_e = float('inf')
    min_z, polarity = 0, 0
    for z, s_m, s_p in zip(zs, s_minuses, s_pluses):
        error_1 = s_p + (t_minus - s_m)
        error_2 = s_m + (t_plus - s_p)
        if error_1 < min_e:
            min_e = error_1
            min_z = z
            polarity = -1
        elif error_2 < min_e:
            min_e = error_2
            min_z = z
            polarity = 1
    return ThresholdPolarity(threshold=min_z, polarity=polarity)


def determine_threshold_polarity(ys: np.ndarray, ws: np.ndarray, zs: np.ndarray) -> ThresholdPolarity:  
    # Sort according to score
    p = np.argsort(zs)
    zs, ys, ws = zs[p], ys[p], ws[p]
    
    # Determine the best threshold: build running sums
    t_minus, t_plus, s_minuses, s_pluses = build_running_sums(ys, ws)
    
    # Determine the best threshold: select optimal threshold.
    return find_best_threshold(zs, t_minus, t_plus, s_minuses, s_pluses) 

In [ ]:
STATUS_EVERY     = 2000
KEEP_PROBABILITY = 1./4.

def build_weak_classifiers(
    prefix: str, num_features: int, xis: np.ndarray, ys: np.ndarray, features: List[fm.Feature], ws: tp.Optional[np.ndarray] = None
) -> Tuple[List[WeakClassifier], List[float]]:
    
    if ws is None:
        # TODO: adapt to 2d weights
        m = len(ys[ys < .5])  # number of negative example
        l = len(ys[ys > .5])  # number of positive examples

        # Initialize the weights
        ws = np.zeros_like(ys)
        ws[ys < .5] = 1./(2.*m)
        ws[ys > .5] = 1./(2.*l)
    
    # Keep track of the history of the example weights.
    w_history = [ws]

    total_start_time = time.time()
    with Parallel(n_jobs=-1, backend='threading') as parallel:
        weak_classifiers = []  # type: List[WeakClassifier]
        for t in range(num_features):
            print(f'Building weak classifier {t+1}/{num_features} ...')
            start_time = time.time()
            
            # Normalize the weights
            #TODO: ignore
            ws = normalize_weights(ws)
            
            status_counter = STATUS_EVERY

            # Select best weak classifier for this round
            best = ClassifierResult(polarity=0, threshold=0, classification_error=float('inf'), classifier=None)
            # TODO: add tqdm
            for i, f in enumerate(features):
                status_counter -= 1
                improved = False

                # Python runs singlethreaded. To speed things up,
                # we're only anticipating every other feature, give or take.
                # TODO: ignore?
                if KEEP_PROBABILITY < 1.:
                    skip_probability = np.random.random()
                    if skip_probability > KEEP_PROBABILITY:
                        continue

                # TODO: avoid
                result = apply_feature(f, xis, ys, ws, parallel)
                
                if result.classification_error < best.classification_error:
                    improved = True
                    best = result

                # Print status every couple of iterations.
                if improved or status_counter == 0:
                    current_time = datetime.now()
                    duration = current_time - start_time
                    total_duration = current_time - total_start_time
                    status_counter = STATUS_EVERY
                    if improved:
                        print(f't={t+1}/{num_features} {total_duration.total_seconds():.2f}s ({duration.total_seconds():.2f}s in this stage) {i+1}/{len(features)} {100*i/len(features):.2f}% evaluated. Classification error improved to {best.classification_error:.5f} using {str(best.classifier)} ...')
                    else:
                        print(f't={t+1}/{num_features} {total_duration.total_seconds():.2f}s ({duration.total_seconds():.2f}s in this stage) {i+1}/{len(features)} {100*i/len(features):.2f}% evaluated.')

            # After the best classifier was found, determine alpha
            beta = best.classification_error / (1 - best.classification_error)
            alpha = np.log(1. / beta)
            
            # Build the weak classifier
            classifier = WeakClassifier(threshold=best.threshold, polarity=best.polarity, classifier=best.classifier, alpha=alpha)
            
            # Update the weights for misclassified examples
            for i, (x, y) in enumerate(zip(xis, ys)):
                h = run_weak_classifier(x, classifier)
                e = np.abs(h - y)
                ws[i] = ws[i] * np.power(beta, 1-e)
                
            # Register this weak classifier           
            weak_classifiers.append(classifier)
            w_history.append(ws)
        
            pickle.dump(classifier, open(f'{prefix}-weak-learner-{t+1}-of-{num_features}.pickle', 'wb'))
    
    print(f'Done building {num_features} weak classifiers.')
    return weak_classifiers, w_history

In [4]:
import typing as tp
PredictionStats = tp.NamedTuple(
    'PredictionStats', [('tn', int), ('fp', int), ('fn', int), ('tp', int)]
)

PredictionStats(*(np.ones(4).ravel()))

<IPython.core.display.Javascript object>

PredictionStats(tn=1.0, fp=1.0, fn=1.0, tp=1.0)